this notebook does some spot checking on learned weight patterns in `scripts/training/yuanyuan_8k_a_3day/feature_approximation/local_pcn_recurrent/submit_sep2.py`.

In [1]:
from thesis_v2 import dir_dict


In [5]:
from sys import path
from os.path import join

In [6]:
folder_to_check = 'scripts/training/yuanyuan_8k_a_3day/feature_approximation/local_pcn_recurrent'
path.insert(0, join(dir_dict['root'], folder_to_check))
from submit_sep2 import good_model_param